In [1]:
import sys
sys.path.append('..')
import ScraperFC as sfc # import local scraperfc
import traceback
from tqdm.auto import tqdm

In [2]:
# # from shared_test_functions import get_random_league_seasons

# # samples = get_random_league_seasons('Oddsportal', 3)

# op = sfc.Oddsportal()
# # fbref = sfc.FBRef()
# counts = dict()
# try:
#     # for year, league in tqdm(samples):
#     league = 'EPL'
#     for year in tqdm(range(2004,2023)):
#         # print('-'*80)
#         # print(year, league)
#         # fbref_links = fbref.get_match_links(year,league)
#         op_links = op.get_match_links(year, league)
#         counts[year] = len(op_links)
#         # break
# except:
#     traceback.print_exc()
# finally:
#     op.close()
#     # fbref.close()

In [6]:
op = sfc.Oddsportal()
try:
    # rand_match_link = links[np.random.randint(0, num_links+1)]
    # rand_match_link = 'https://oddsportal.com/football/england/premier-league-2005-2006/chelsea-birmingham-be9APe6e' # no O/U odds
    # rand_match_link = 'https://www.oddsportal.com/football/england/premier-league/west-ham-southampton-tfnoGQAd' # weird date format
    rand_match_link = 'https://www.oddsportal.com/football/england/premier-league/aston-villa-bournemouth-Kdi8EDZi/' # everything should work
    # match = op.scrape_match(rand_match_link)

    moneyline_df = op.get_1X2odds_from_match(rand_match_link)
    over_under_df = op.get_OUodds_from_match(rand_match_link)
except:
    traceback.print_exc()
finally:
    op.close()

In [7]:
over_under_df

,GGBET +0.25 over,GGBET +0.25 under,GGBET +0.25 po %,Average +0.25 over,Average +0.25 under,Average +0.25 po %,Highest +0.25 over,Highest +0.25 under,Highest +0.25 po %,10x10bet +0.5 over,...,1xBet +7.5 po %,bet365 +7.5 over,bet365 +7.5 under,bet365 +7.5 po %,Average +7.5 over,Average +7.5 under,Average +7.5 po %,Highest +7.5 over,Highest +7.5 under,Highest +7.5 po %
0,-5000.0,1096.0,94.0,-5000.0,1096.0,94.0,-5000.0,1096.0,94.0,-1667.0,...,98.1,5000.0,NaN,98.1,5000.0,NaN,98.1,5000.0,NaN,98.1
